In [1]:
%cd /content/drive/MyDrive/ugs-applications/

/content/drive/MyDrive/ugs-applications


In [2]:
!pip3 install transformer_lens
!pip3 install seaborn
!pip3 install fancy_einsum
!pip3 install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.8 MB/s eta 0:00:00
  Using cached nvid

In [3]:
import torch
import numpy as np
from tqdm import tqdm
from fancy_einsum import einsum
from einops import rearrange
import math
from functools import partial
import torch.optim
import time
from itertools import cycle
import os
import seaborn as sns
import argparse
import matplotlib.pyplot as plt
import pickle
from utils.training_utils import load_model_data, load_args, update_means_variances, plot_no_outliers
from utils.MaskConfig import VertexInferenceConfig
from utils.task_datasets import get_task_ds
from pruners.VertexPruner import VertexPruner
from mask_samplers.AblationMaskSampler import SingleComponentMaskSampler

In [4]:
model_name = "gpt2-small"
owt_batch_size = 10
device, model, tokenizer, owt_iter = load_model_data(model_name, owt_batch_size)
model.eval()
# model.cfg.use_attn_result = True
n_layers = model.cfg.n_layers
n_heads = model.cfg.n_heads

Loading model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
Loading OWT...
Loading OWT data from disk
Making DataLoader


In [5]:
# desc: ablation type. Supported ablation types: zero, mean, oa, resample, cf_mean, cf
args = load_args("ablation_loss", defaults={"desc": "mean_agnostic", "dataset": "ioi"})
folder, ablation_type, dataset = args["folder"], args["desc"], args["dataset"]

pruning_cfg = VertexInferenceConfig(model.cfg, device, folder, init_param=1)
pruning_cfg.batch_size = 10

oa_train = False
if ablation_type.startswith("oa"):
    if os.path.exists(f"{folder}/{ablation_type}_modes.pth"):
        init_modes = torch.load(f"{folder}/{ablation_type}_modes.pth")
    else:
        pruning_cfg.batch_size = 3
        oa_train = True

# fix_prompt: only resample from the same prompt
task_ds = get_task_ds(dataset, pruning_cfg.batch_size, device, ablation_type)

for param in model.parameters():
    param.requires_grad = False


Resetting to default parameters
None


usage: colab_kernel_launcher.py [-h] [-l LAMB] [-d DATASET] [-s SUBFOLDER] [-c PRIORSCALE]
                                [-p PRIORLAMB] [-n NAME] [-t TAU] [-e DESC]
                                [-w | --window | --no-window] [--minwindow MINWINDOW]
                                [--maxwindow MAXWINDOW]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-648549a9-d5ad-4123-9b90-6cbda5ac8177.json


In [6]:
print("Folder: {}".format(folder))
print("Ablation type: {}".format(ablation_type))
print("Dataset: {}".format(dataset))
print("OA Train: {}".format(oa_train))

Folder: results/ablation_loss/ioi
Ablation type: mean_agnostic
Dataset: ioi
OA Train: False


In [7]:
print("Pruning Config")
print("-------------------------------------")
for name, value in pruning_cfg.__dict__.items():
  if name.startswith("init_params"):
    continue
  print(f"{name}: {value}")


Pruning Config
-------------------------------------
device: cuda:0
n_layers: 12
n_heads: 12
folder: results/ablation_loss/ioi
lamb: None
record_every: 100
checkpoint_every: 5
starting_beta: 0.6666666666666666
hard_concrete_endpoints: (-0.1, 1.1)
layers_to_prune: [('attn', 0), ('mlp', 0), ('attn', 1), ('mlp', 1), ('attn', 2), ('mlp', 2), ('attn', 3), ('mlp', 3), ('attn', 4), ('mlp', 4), ('attn', 5), ('mlp', 5), ('attn', 6), ('mlp', 6), ('attn', 7), ('mlp', 7), ('attn', 8), ('mlp', 8), ('attn', 9), ('mlp', 9), ('attn', 10), ('mlp', 10), ('attn', 11), ('mlp', 11), ('mlp', 12)]
temp_min_reg: 0.001
temp_adj_intv: 10
temp_avg_intv: 20
temp_comp_intv: 200
temp_convergence_target: 2000
temp_c: 0
temp_momentum: 0
constant_prune_mask: {'attn': [tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0'), tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0'), tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0'), tensor([[1., 1., 1., 1.

In [8]:
print("Task DS Config")
print("-------------------------------------")
for name,value in task_ds.__dict__.items():
  print(f"{name}: {value}")

Task DS Config
-------------------------------------
ds_name: ioi
batch_size: 10
device: cuda:0
ablation_type: mean_agnostic
means: None
ds: <list_iterator object at 0x7e8041226800>
seed: 0
fix_prompt: False


In [9]:
mask_sampler = SingleComponentMaskSampler(pruning_cfg)
pruning_cfg.n_samples = mask_sampler.n_components

pruner_args = task_ds.get_pruner_args({"zero", "mean", "resample", "cf_mean", "cf", "oa", "oa_specific","mean_agnostic"})


if ablation_type.startswith("oa") and not oa_train:
    pruner_args['init_modes'] = init_modes['modal_attention'], init_modes['modal_mlp']

vertex_pruner = VertexPruner(model, pruning_cfg, mask_sampler, **pruner_args)
vertex_pruner.add_patching_hooks()

In [11]:
mask_sampler()
mask_sampler.sampled_mask["mlp"][0].shape

torch.Size([1560])

In [ ]:
print("Mask Sampler Config")
print("-------------------------------------")
for name,value in mask_sampler.__dict__.items():
  print(f"{name}: {value}")
mask_sampler.sampled_mask["attn"][0].shape

Mask Sampler Config
-------------------------------------
training: True
_parameters: OrderedDict()
_buffers: OrderedDict()
_non_persistent_buffers_set: set()
_backward_pre_hooks: OrderedDict()
_backward_hooks: OrderedDict()
_is_full_backward_hook: None
_forward_hooks: OrderedDict()
_forward_hooks_with_kwargs: OrderedDict()
_forward_hooks_always_called: OrderedDict()
_forward_pre_hooks: OrderedDict()
_forward_pre_hooks_with_kwargs: OrderedDict()
_state_dict_hooks: OrderedDict()
_state_dict_pre_hooks: OrderedDict()
_load_state_dict_pre_hooks: OrderedDict()
_load_state_dict_post_hooks: OrderedDict()
_modules: OrderedDict()
use_temperature: False
log_columns: []
n_components: 156
sampled_mask: {'attn': [tensor([[0., 1., 1.,  ..., 1., 1., 1.],
        [0., 1., 1.,  ..., 1., 1., 1.],
        [0., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0'), tensor([[1., 1., 1

torch.Size([1560, 12])

In [ ]:
if oa_train:
    max_batches = 10000
    modal_optimizer = torch.optim.AdamW([vertex_pruner.modal_attention, vertex_pruner.modal_mlp], lr=pruning_cfg.lr_modes, weight_decay=0)
else:
    max_batches = 10000 // pruning_cfg.batch_size
    head_losses = torch.zeros((pruning_cfg.n_samples,1)).to(device)
    head_vars = torch.zeros((pruning_cfg.n_samples,1)).to(device)

In [ ]:
head_losses.shape

torch.Size([156, 1])

In [ ]:
def save_snapshot(head_losses, head_vars):
    torch.save({"head_losses": head_losses, "head_vars": head_vars}, f"{folder}/{ablation_type}_results.pth")

    plot_no_outliers(
        sns.scatterplot, .03,
        head_losses, head_vars.sqrt(),
        args={"x": "Component mean loss", "y": "Component std loss", "s": 5,
              "f": f"{folder}/{ablation_type}.png"}
    )

In [ ]:
for no_batches in tqdm(range(max_batches)):
    batch, last_token_pos, cf = task_ds.retrieve_batch_cf(tokenizer)

    if oa_train:
        modal_optimizer.zero_grad()

        loss = vertex_pruner(batch, last_token_pos)
        loss.backward()
        modal_optimizer.step()

        if no_batches % -100 == -1:
            torch.save({"modal_attention": vertex_pruner.modal_attention, "modal_mlp": vertex_pruner.modal_mlp}, f"{folder}/{ablation_type}_modes.pth")
            vertex_pruner.log.plot(["kl_loss"], mv=100, save=f"{folder}/{ablation_type}_train.png")
    else:
        with torch.no_grad():
            # loss: [n_components, batch_size]
            loss, _ = vertex_pruner(batch, last_token_pos, counterfactual=cf, separate_loss=True)
            head_losses, head_vars = update_means_variances(head_losses, head_vars, loss, no_batches)

        if no_batches % -100 == -1:
            save_snapshot(head_losses, head_vars)

if not oa_train:
    save_snapshot(head_losses, head_vars)

  0%|          | 1/1000 [00:02<34:13,  2.06s/it]

Cuda time 0.449535995721817
Cuda time 532.5260620117188
Cuda time 0.0030720001086592674
KL: 0.046690434217453


  0%|          | 2/1000 [00:02<19:00,  1.14s/it]

Cuda time 0.2908160090446472
Cuda time 497.3465576171875
Cuda time 0.004095999989658594
KL: 0.048396531492471695


  0%|          | 3/1000 [00:03<14:07,  1.18it/s]

...

Cuda time 0.26214399933815
Cuda time 495.79315185546875
Cuda time 0.0030720001086592674
KL: 0.040480706840753555


100%|█████████▉| 999/1000 [08:39<00:00,  1.99it/s]

Cuda time 0.27136000990867615
Cuda time 495.13677978515625
Cuda time 0.0030720001086592674
KL: 0.0382121317088604


100%|██████████| 1000/1000 [08:39<00:00,  1.92it/s]

Cuda time 0.25088000297546387
Cuda time 495.46136474609375
Cuda time 0.0030720001086592674
KL: 0.04885804280638695


In [ ]:
head_vars.shape

torch.Size([156, 1])